# Introduction to Natural Language Processing 01
## Lab 03
Nelson VICEL--FARAH\
Karen KASPAR\
Romain BRAND

In [1]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")

imdb_dataset

Reusing dataset imdb (/home/miolith/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [8]:
import pandas as pd
import numpy as np
from string import punctuation

def preparing(df):
    #change values 0 and 1 to negative and positive
    df['label'] = df['label'].replace([0, 1], ['negative', 'positive'])

    #shuffle the dataset so that we don't have all the positive rows followed by only negative rows
    df = df.sample(frac=1).reset_index()

    #pretreatment (applying lower case and punctuation removal) 
    df['text'] = df['text'].str.lower()
    df['text'] = df['text'].str.replace('[{}]'.format(punctuation), ' ', regex=True)

def save_to_fasttext(df, filename):
    #save the content of the dataframe into a text file with the appropriate format
    np.savetxt(filename, '__label__'+df['label']+' '+df['text'], fmt='%s')

In [14]:
train_df = pd.DataFrame(imdb_dataset['train'])
test_df = pd.DataFrame(imdb_dataset['test'])


save_to_fasttext(preparing(train_df), "train_df.txt")
save_to_fasttext(preparing(test_df), "test_df.txt")

In [11]:
!head -n 2 'train_df.txt'

__label__negative i think  the best of times  was a lost cause from the get go  the initial premise  guy drops the winning touchdown pass against a rival high school team  can never seem to get over it and then tries to reunite the two teams to play again  is one of the dumbest i have ever heard  since ron shelton went on to write much better sports films i wonder if there was more to it then that  i hope this film wasn t green lit with shelton pitching the story as i wrote above  br    br   so we have the premise  going from there you would think  or hope  that there might be a few twists along the way to keep things lively  no such luck  this script follows every predictable cliché you can think of  there isn t a moment in this film you won t see coming a mile away before the film reveals it and the ending     well if you can t figure out the ending by the end of the first reel then you haven t paid attention or seen any other sports movie in your life  br    br   robin williams and 

In [12]:
import fasttext

In [13]:
model = fasttext.train_supervised('train_df.txt')

Read 6M words
Number of words:  75910
Number of labels: 2
Progress: 100.0% words/sec/thread: 3080606 lr:  0.000000 avg.loss:  0.389864 ETA:   0h 0m 0s


In [15]:
model.test("test_df.txt")

(25000, 0.8786, 0.8786)

In [31]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train_df, shuffle=True)

save_to_fasttext(preparing(train), "cooking.train")
save_to_fasttext(preparing(val), "cooking.valid")

In [32]:
model = fasttext.train_supervised(input='cooking.train', autotuneValidationFile='cooking.valid')

Progress: 100.0% Trials:    9 Best score:  0.894240 ETA:   0h 0m 0s
Training again with best arguments
Read 4M words
Number of words:  67664
Number of labels: 2
Progress: 100.0% words/sec/thread: 1793476 lr:  0.000000 avg.loss:  0.047440 ETA:   0h 0m 0s


In [33]:
model.test("test_df.txt")

(25000, 0.89396, 0.89396)

In [73]:
test_df["prediction"] = test_df["text"].apply(lambda x: int("p" in model.predict(x)[0][0]))
test_df

,text,label,prediction
0,I love sci-fi and am willing to put up with a ...,0,0
1,"Worth the entertainment value of a rental, esp...",0,1
2,its a totally average film with a few semi-alr...,0,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0,0
4,"First off let me say, If you haven't enjoyed a...",0,1
...,...,...,...
24995,Just got around to seeing Monster Man yesterda...,1,1
24996,I got this as part of a competition prize. I w...,1,1
24997,I got Monster Man in a box set of three films ...,1,1
24998,"Five minutes in, i started to feel how naff th...",1,0


In [80]:
failed_sample1 = test_df[(test_df["prediction"] == 1) & (test_df["label"] == 0)].iloc[1,:]["text"]
failed_sample2 = test_df[(test_df["prediction"] == 0) & (test_df["label"] == 1)].iloc[2,:]["text"]

### Cet exemple est un faux positif.

In [78]:
failed_sample1

"First off let me say, If you haven't enjoyed a Van Damme movie since bloodsport, you probably will not like this movie. Most of these movies may not have the best plots or best actors but I enjoy these kinds of movies for what they are. This movie is much better than any of the movies the other action guys (Segal and Dolph) have thought about putting out the past few years. Van Damme is good in the movie, the movie is only worth watching to Van Damme fans. It is not as good as Wake of Death (which i highly recommend to anyone of likes Van Damme) or In hell but, in my opinion it's worth watching. It has the same type of feel to it as Nowhere to Run. Good fun stuff!"

### Cet exemple est un faux négatif

In [81]:
failed_sample2

"It is more a subtle story of the fact that in Indian household how most decisions are taken by the man, how no attention is paid to the desires of the lady, for example how even when the husband and wife sleep together it would be a test for the husband whether he can control his desires, not to give the woman pleasure. And in such a type of scenario, women invariably have two choices, either to accept all this and take it into their own life, which is usually the case or not accept this and try to mould things to satisfy themselves, which makes a movie!<br /><br />Fire is a brilliantly directed story of the second option, which women choose for themselves, no sacrifice, not to serve anybody else, rather a decision for their own good. Somehow the whole idea of justifying lesbianism didn't find an acceptance in the Indian audience but if one looks the whole movie from an angle of self-expression, then the whole debate doesn't even arise."